In [1]:
# Example 4-1. Load the Tox21 dataset
import deepchem as dc
_, (train, valid, test), _ = dc.molnet.load_tox21()
train_X, train_y, train_w = train.X, train.y, train.w
valid_X, valid_y, valid_w = valid.X, valid.y, valid.w
test_X, test_y, test_w = test.X, test.y, test.w

Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.


In [2]:
# Example 4-2. Remove extra datasets from Tox21
# Remove extra tasks
train_y = train_y[:, 0]
valid_y = valid_y[:, 0]
test_y = test_y[:, 0]
train_w = train_w[:, 0]
valid_w = valid_w[:, 0]
test_w = test_w[:, 0]

In [3]:
# # Example 4-3. Defining placeholders that accept minibatches of different sizes
# import tensorflow as tf
# d = 1024
# with tf.name_scope("placeholders"):
#     x = tf.placeholder(tf.float32, (None, d))
#     y = tf.placeholder(tf.float32, (None,))

In [4]:
# # Example 4-4. Defining a hidden layer
# n_hidden = 256
# with tf.name_scope("hidden-layer"):
#     W = tf.Variable(tf.random_normal((d, n_hidden)))
#     b = tf.Variable(tf.random_normal((n_hidden,)))
#     x_hidden = tf.nn.relu(tf.matmul(x, W) + b)

In [6]:
# Example 4-5. Defining the fully connected architecture
import tensorflow as tf
d = 1024
n_hidden = 256
learning_rate = 1e-3

with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (None, d))
    y = tf.placeholder(tf.float32, (None,))

with tf.name_scope("hidden-layer"):
    W = tf.Variable(tf.random_normal((d, n_hidden)))
    b = tf.Variable(tf.random_normal((n_hidden,)))
    x_hidden = tf.nn.relu(tf.matmul(x, W) + b)

with tf.name_scope("output"):
    W = tf.Variable(tf.random_normal((n_hidden, 1)))
    b = tf.Variable(tf.random_normal((1,)))
    y_logit = tf.matmul(x_hidden, W) + b
    # the sigmoid gives the class probability of 1
    y_one_prob = tf.sigmoid(y_logit)
    # Rounding P(y=1) will give the correct prediction.
    y_pred = tf.round(y_one_prob)

with tf.name_scope("loss"):
    # Compute the cross-entropy term for each datapoint
    y_expand = tf.expand_dims(y, 1)
    entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y_expand)
    # Sum all contributions
    l = tf.reduce_sum(entropy)

with tf.name_scope("optim"):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(l)

with tf.name_scope("summaries"):
    tf.summary.scalar("loss", l)
    merged = tf.summary.merge_all()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
# Example 4-6. Add a placeholder for dropout probability
keep_prob = tf.placeholder(tf.float32)

In [8]:
# Example 4-7. Defining a hidden layer with dropout
with tf.name_scope("hidden-layer"):
    W = tf.Variable(tf.random_normal((d, n_hidden)))
    b = tf.Variable(tf.random_normal((n_hidden,)))
    x_hidden = tf.nn.relu(tf.matmul(x, W) + b)
    # Apply dropout
    x_hidden = tf.nn.dropout(x_hidden, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
# Example 4-8. Training on minibatches
n_epochs = 500
step = 0
dropout_prob = 0.1
N = len(train_X)
batch_size = 250
for epoch in range(n_epochs):
    pos = 0
    while pos < N:
        batch_X = train_X[pos:pos+batch_size]
        batch_y = train_y[pos:pos+batch_size]
        feed_dict = {x: batch_X, y: batch_y, keep_prob: dropout_prob}
        _, summary, loss = sess.run([train_op, merged, l], feed_dict=feed_dict)
        print("epoch %d, step %d, loss: %f" % (epoch, step, loss))
        train_writer.add_summary(summary, step)
        step += 1
        pos += batch_size

NameError: name 'sess' is not defined